In [4]:
import google.generativeai as genai
import yaml
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
GEMINI_API_KEY = config['GEMINI_API_KEY']
genai.configure(api_key=GEMINI_API_KEY)

In [5]:
def save_file(filename: str, content: str) -> str:
    """
    Saves raw text in the content string to a file specified by filename string
    """
    with open(filename, 'w') as f:
        f.write(content)

In [6]:
PROMPT = """
<Task> Design a SqlLite database for my university course website </Task>
<SubTasks> 
    <1> Decide a suitable database schema for my use-case. Think step by step for my requirements, and identify important attributes for my use case.
        We need to store lecture no, description of lecture, course title, and lecture title, and we need a single database that will work for all my current and future courses. </1>
    <2> After deciding upon schema, you will have to use SqlLite schema to generate the database. Add this to your python scripty </2>
    <3> Think hard for good design pattern and right way before creating the database, you need to populate sample data in the database as well. For this make sure to add this to script for this as well </3>
    <4> Write a final python script that will do the required tasks, save it using save_file tool.</4>
</SubTasks>
<Tools>save_file</Tools>
"""

In [7]:
model = genai.GenerativeModel('gemini-2.0-flash-exp',tools=[save_file])
chat = model.start_chat(enable_automatic_function_calling=True)
chat.send_message(PROMPT)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The Python script has been saved to `course_database.py`. You can now run this script to create the SQLite database `course_website.db` and populate it with the sample data.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.10550079113099634
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 2202,
        "candidates_token_count": 41,
        "total_token_count": 2243
      }
    }),
)

In [85]:
!python create_university_database.py

Database created and populated successfully.


In [1]:
# In this cell, we extract the schema from a db file along with field names as well as the exact commands that create the table to extract its properties
import sqlite3
import json

dbname = 'course_website.db'
output_file = 'db_schema.json'

# Connect to the database and extract schema
with sqlite3.connect(dbname) as con:
    cursor = con.cursor()
    cursor.execute("SELECT name, type, sql FROM sqlite_master WHERE type IN ('table', 'index', 'view')")
    
    schema_info = []
    for row in cursor.fetchall():
        schema_info.append({
            "name": row[0],  # Table, Index, or View name
            "type": row[1],  # 'table', 'index', or 'view'
            "sql": row[2]    # SQL statement to create the object
        })

    cursor.close()

schema_info


[{'name': 'courses',
  'type': 'table',
  'sql': 'CREATE TABLE courses (\n            course_id INTEGER PRIMARY KEY AUTOINCREMENT,\n            course_title TEXT NOT NULL\n        )'},
 {'name': 'sqlite_sequence',
  'type': 'table',
  'sql': 'CREATE TABLE sqlite_sequence(name,seq)'},
 {'name': 'lectures',
  'type': 'table',
  'sql': 'CREATE TABLE lectures (\n            lecture_id INTEGER PRIMARY KEY AUTOINCREMENT,\n            course_id INTEGER NOT NULL,\n            lecture_no INTEGER,\n            lecture_title TEXT NOT NULL,\n            lecture_description TEXT,\n            FOREIGN KEY (course_id) REFERENCES courses (course_id)\n        )'}]

In [2]:
# Connect to the database
with sqlite3.connect(dbname) as con:
    cursor = con.cursor()
    
    # Get all courses
    print("All Courses:")
    cursor.execute("SELECT * FROM courses")
    courses = cursor.fetchall()
    for course in courses:
        print(f"Course ID: {course[0]}, Title: {course[1]}")
    
    print("\nAll Lectures:")
    # Get all lectures
    cursor.execute("SELECT * FROM lectures")
    lectures = cursor.fetchall()
    for lecture in lectures:
        print(f"Lecture ID: {lecture[0]}, Course ID: {lecture[1]}, " 
              f"Lecture No: {lecture[2]}, Title: {lecture[3]}, "
              f"Description: {lecture[4]}")
    
    print("\nLectures for Course ID 1:")
    # Get lectures for course_id = 1
    cursor.execute("SELECT * FROM lectures WHERE course_id = 1")
    course_1_lectures = cursor.fetchall()
    for lecture in course_1_lectures:
        print(f"Lecture ID: {lecture[0]}, Course ID: {lecture[1]}, "
              f"Lecture No: {lecture[2]}, Title: {lecture[3]}, "
              f"Description: {lecture[4]}")


All Courses:
Course ID: 1, Title: Introduction to Computer Science
Course ID: 2, Title: Data Structures and Algorithms

All Lectures:
Lecture ID: 1, Course ID: 1, Lecture No: 1, Title: Introduction to Programming, Description: Overview of programming concepts.
Lecture ID: 2, Course ID: 1, Lecture No: 2, Title: Data Types and Variables, Description: Understanding different data types and variables.
Lecture ID: 3, Course ID: 2, Lecture No: 1, Title: Introduction to Data Structures, Description: Overview of data structures.
Lecture ID: 4, Course ID: 2, Lecture No: 2, Title: Arrays and Linked Lists, Description: Detailed explanation of arrays and linked lists.

Lectures for Course ID 1:
Lecture ID: 1, Course ID: 1, Lecture No: 1, Title: Introduction to Programming, Description: Overview of programming concepts.
Lecture ID: 2, Course ID: 1, Lecture No: 2, Title: Data Types and Variables, Description: Understanding different data types and variables.


In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

# Create engine and session
engine = create_engine('sqlite:///course_website.db')
Session = sessionmaker(bind=engine)
session = Session()

# Define models
Base = declarative_base()

class Course(Base):
    __tablename__ = 'courses'
    
    course_id = Column(Integer, primary_key=True, autoincrement=True)
    course_title = Column(String, nullable=False)
    lectures = relationship("Lecture", back_populates="course")

class Lecture(Base):
    __tablename__ = 'lectures'
    
    lecture_id = Column(Integer, primary_key=True, autoincrement=True)
    course_id = Column(Integer, ForeignKey('courses.course_id'), nullable=False)
    lecture_no = Column(Integer)
    lecture_title = Column(String, nullable=False)
    lecture_description = Column(String)
    course = relationship("Course", back_populates="lectures")

# Query the database
print("All Courses:")
courses = session.query(Course).all()
for course in courses:
    print(f"Course ID: {course.course_id}, Title: {course.course_title}")

print("\nAll Lectures:")
lectures = session.query(Lecture).all()
for lecture in lectures:
    print(f"Lecture ID: {lecture.lecture_id}, Course ID: {lecture.course_id}, "
          f"Lecture No: {lecture.lecture_no}, Title: {lecture.lecture_title}, "
          f"Description: {lecture.lecture_description}")

print("\nLectures for Course ID 1:")
course_1_lectures = session.query(Lecture).filter(Lecture.course_id == 1).all()
for lecture in course_1_lectures:
    print(f"Lecture ID: {lecture.lecture_id}, Course ID: {lecture.course_id}, "
          f"Lecture No: {lecture.lecture_no}, Title: {lecture.lecture_title}, "
          f"Description: {lecture.lecture_description}")

session.close()


All Courses:
Course ID: 1, Title: Introduction to Computer Science
Course ID: 2, Title: Data Structures and Algorithms

All Lectures:
Lecture ID: 1, Course ID: 1, Lecture No: 1, Title: Introduction to Programming, Description: Overview of programming concepts.
Lecture ID: 2, Course ID: 1, Lecture No: 2, Title: Data Types and Variables, Description: Understanding different data types and variables.
Lecture ID: 3, Course ID: 2, Lecture No: 1, Title: Introduction to Data Structures, Description: Overview of data structures.
Lecture ID: 4, Course ID: 2, Lecture No: 2, Title: Arrays and Linked Lists, Description: Detailed explanation of arrays and linked lists.

Lectures for Course ID 1:
Lecture ID: 1, Course ID: 1, Lecture No: 1, Title: Introduction to Programming, Description: Overview of programming concepts.
Lecture ID: 2, Course ID: 1, Lecture No: 2, Title: Data Types and Variables, Description: Understanding different data types and variables.


/var/folders/13/p2nn0qmx63b6w_67cmcm1y7w0000gp/T/ipykernel_84387/4154301665.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
